# Merge tables from Tobin et al. (2020)

In [1]:
import numpy as np
import astropy.table

In [2]:
# convert -99 (used to lable nan in table) to masked
def mark_nan(table_in,col_name):
    a = np.ma.MaskedArray(table_in[col_name], fill_value=np.nan)
    a.mask = np.array([False]*len(a))
    a.mask[a==-99] = True
    table_in[col_name] = a
    return table_in

# create new columns for RA and DEC
def get_RA_DEC(table_in):
    RA_deg = (table_in['RAh']+table_in['RAm']/60+table_in['RAs']/3600)*(360/24)
    DEC_deg = (table_in['DEd']+table_in['DEm']/60+table_in['DEs']/3600)*(1*(table_in['DE-']=='+')-1*(table_in['DE-']=='-'))
    table_in['RA_deg'] = RA_deg
    table_in['DEC_deg'] = DEC_deg
    return table_in

# do both operations above
def format_T20_table(table_in):
    for col_name in table_in.columns:
        if table_in[col_name].dtype == np.dtype('float64'):
            table_in = mark_nan(table_in,col_name)
    if 'RAh' in table_in.columns:
        table_in = get_RA_DEC(table_in)
    return table_in

In [3]:
field_alma = format_T20_table(astropy.table.Table.read("../data/Tobin_et_al_2020_tables/apjab6f64t2_mrt.txt", format="ascii.cds"))
field_vla = format_T20_table(astropy.table.Table.read("../data/Tobin_et_al_2020_tables/apjab6f64t4_mrt.txt", format="ascii.cds"))
data_alma = format_T20_table(astropy.table.Table.read("../data/Tobin_et_al_2020_tables/apjab6f64t6_mrt.txt", format="ascii.cds"))
data_vla = format_T20_table(astropy.table.Table.read("../data/Tobin_et_al_2020_tables/apjab6f64t7_mrt.txt", format="ascii.cds"))
data_derived = format_T20_table(astropy.table.Table.read("../data/Tobin_et_al_2020_tables/apjab6f64t8_mrt.txt", format="ascii.cds"))

In [4]:
# drop unecessary columns

# fields
to_keep = ['Field','Distance']
to_drop = list(set(field_alma.columns)-set(to_keep))
field_alma.remove_columns(to_drop)
field_vla.remove_columns(to_drop)
field_alma.rename_column('Field','FieldA')
field_alma.rename_column('Distance','DistanceA')
field_vla.rename_column('Field','FieldV')
field_vla.rename_column('Distance','DistanceV')

# alma / vla data
to_drop = ['RAh','RAm','RAs','DE-','DEd','DEm','DEs','DistFA','DistFV','Class','Flag']
data_alma.remove_columns(to_drop+['FieldV'])
data_vla.remove_columns(to_drop+['FieldA'])
for col in list(data_alma.columns):
    if col not in ['Source','FieldA','FieldV']:
        data_alma.rename_column(col, 'A_'+col)
for col in list(data_vla.columns):
    if col not in ['Source','FieldA','FieldV']:
        data_vla.rename_column(col, 'V_'+col)

In [5]:
# joining the tables
data_alma_w_dist = astropy.table.join(data_alma,field_alma,join_type='left',keys='FieldA')
data_vla_w_dist  = astropy.table.join(data_vla ,field_vla ,join_type='left',keys='FieldV')
data_all = astropy.table.join(data_derived, data_alma_w_dist, join_type='left',keys='Source')
data_all = astropy.table.join(data_all, data_vla_w_dist, join_type='left',keys='Source')
data_all = astropy.table.unique(data_all, silent=True) # remove duplicate rows
data_all.add_index('Source') # add index by source

In [6]:
# print out a few rows in the middle
i0 = data_all.loc_indices['HOPS-409']
data_all[i0-3:i0+3]

Source,RAh,RAm,RAs,DE-,DEd,DEm,DEs,Lbol,Tbol,Class,RdiskA,e_RdiskA,RdiskV,e_RdiskV,MdiskA,e_MdiskA,MdiskV,e_MdiskV,QALMA,QVLA,SpIdxAV,e_SpIdxAV,SpIdxV,e_SpIdxV,Flag,RA_deg,DEC_deg,FieldA,A_Flux,A_e_Flux,A_PFlux,A_RMS,A_dBmaj,A_dBmin,A_dPA,A_RA_deg,A_DEC_deg,DistanceA,FieldV,V_Flux,V_e_Flux,V_PFlux,V_RMS,V_dBmaj,V_dBmin,V_dPA,V_RA_deg,V_DEC_deg,DistanceV
,h,min,s,,deg,arcmin,arcsec,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str21,int64,int64,float64,str1,int64,int64,float64,float64,float64,str4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,str10,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
HOPS-406,5,47,43.434,+,0,38,21.31,0.5,24.6,0,51.5,10.0,--,--,24.3,1.1,81.0,17.8,29.9,--,2.5,0.11,5.4,2.37,dd,86.93097499999999,0.6392527777777778,HOPS-406,15.471,0.694,8.174,0.21,0.14,0.11,64.2,86.93097499999999,0.6392527777777778,430.3,HOPS-406,0.041,0.009,0.044,8.51,0.0,0.0,0.0,86.93097916666666,0.6392583333333333,430.3
HOPS-407,5,46,28.251,+,0,19,27.96,0.7,26.8,0,64.1,10.0,35.6,10.0,83.2,2.4,238.4,30.9,7.43,2.81,2.6,0.08,1.5,1.17,dd,86.6177125,0.3244333333333333,HOPS-407,62.196,1.762,24.426,0.43,0.18,0.14,173.6,86.6177125,0.3244333333333333,419.1,HOPS-407,0.139,0.018,0.096,6.9,0.1,0.04,162.2,86.6177125,0.3244388888888889,419.1
HOPS-408,5,39,30.904,-,7,23,59.67,0.5,37.9,0,93.5,10.0,22.0,10.0,124.5,3.6,364.1,29.4,5.11,2.22,2.6,0.07,1.8,0.69,dd,84.87876666666668,-7.399908333333333,HOPS-408,90.365,2.492,20.64,0.37,0.28,0.24,157.5,84.87876666666668,-7.399908333333333,398.9,HOPS-408,0.211,0.017,0.154,7.22,0.07,0.05,141.4,84.87877083333333,-7.399908333333333,398.9
HOPS-409,5,35,21.363,-,5,13,17.83,7.2,28.4,0,112.1,10.0,54.7,10.0,110.7,1.1,684.8,31.0,2.44,0.5,2.3,0.06,2.5,0.4,dd,83.8390125,-5.221619444444444,HOPS-409,185.41,1.9,39.275,0.36,0.34,0.12,115.8,83.8390125,-5.221619444444444,392.8,HOPS-409,0.818,0.037,0.279,9.5,0.16,0.06,108.5,83.83901666666667,-5.221616666666667,392.8
HOPS-41,5,34,29.463,-,5,35,42.8,1.7,82.3,I,13.3,10.0,--,--,23.3,0.5,--,--,29.29,--,--,--,--,--,dn,83.62276250000001,-5.595222222222222,HOPS-41,25.979,0.555,23.025,0.29,0.04,0.03,45.5,83.62276250000001,-5.595222222222222,390.9,--,--,--,--,--,--,--,--,--,--,--
HOPS-42,5,35,5.061,-,5,35,40.79,0.2,200.9,I,133.0,10.0,--,--,80.2,2.5,42.5,--,8.89,--,--,--,--,--,du,83.7710875,-5.594663888888888,HOPS-42,48.731,1.507,9.843,0.27,0.4,0.1,106.2,83.7710875,-5.594663888888888,391.7,HOPS-43,--,--,--,9.84,--,--,--,83.7710875,-5.594663888888888,391.7


In [7]:
# save the table
data_all.write("../data/VANDAM_T20_properties.txt", format="ascii")

# Sample selection

In [8]:
data = data_all
source_list = []
n_row = len(data['Source'])
for i in range(n_row):
    # requirements:
    # (1) systems observed by both ALMA and VLA
    # (2)finite dconvolved size (required for inclination estimate)
    flag = data['Flag'][i] == 'dd' and data['A_dBmin'][i]>0 and data['A_dBmaj'][i]>0
    if flag: source_list.append(data['Source'][i])
print(len(source_list), 'sources in list')

163 sources in list


A few of these sources will be excluded from our sample because the corresponding images are not avilable at https://dataverse.harvard.edu/dataverse/VANDAMOrion

In [9]:
# save the list of systems; this preserves the order and
# makes it easier to re-draw the same random subsamples
# in the future.
np.save('../data/Xu22_sources.npy',source_list)

# Download images and fit model to images

In [10]:
import download_vandam_image

In [11]:
# example: download one image
_=download_vandam_image.download_image('HOPS-409',obs='both',folder='../data/observation',verbose=True)

file name: HOPS-409_cont_robust0.5.pbcor.fits
file already exists, skipping download
file name: HOPS-409.A.Ka.cont.0.5.robust.image.pbcor.fits
file already exists, skipping download


In [12]:
# If you want to re-do the fittings, download images and fit the model
# to the images using the commented code below.
# Otherwise, run download_fitted_systems.py to download pkl files
# containing fitted models to ../data/fitted_systems.

pkl files containing fitted models are avilable at
# ../data/fitted_systems.

# uncomment this code block to download all images in the source list
"""
for s in source_list:
    suc = download_vandam_image.download_image(
        s,obs='both',
        folder='../data/observation',
        data_table='../data/VANDAM_T20_properties.txt',
        verbose=False)
    if suc:
        print('success:',s)
    else:
        print('-FAILED:',s)
"""
# uncomment this code block to fit the model for all images
"""
from disk_model import fit_vandam_image
Ds = []
for s in source_list:
    D = fit_vandam_image(s, data_folder_path='../data')
    Ds.append(D)
"""

"\nfor s in source_list:\n    suc = download_vandam_image.download_image(s,obs='both',folder='../data/observation',verbose=False)\n    if suc:\n        print('success:',s)\n    else:\n        print('-FAILED:',s)\n"